In [ ]:
# !pip install cryptography
# !pip install PyMySQL

In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.debug = True
app.config['SQLALCHEMY_DATABASE_URI'] = 'mysql+pymysql://{username}:{password}@{host}:{port}/{db}?charset=utf8'.format(username="kokon",
            password="TrOlOlO123))", host="127.0.0.1", port=3306, db="prediction")
db = SQLAlchemy(app)


In [2]:
class Subject(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)

class Results(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    succes_done = db.Column(db.Boolean, default=False)
    score = db.Column(db.Float, default=0.0)
    comands_id = db.Column(db.Integer, db.ForeignKey('comands.id'), nullable=False)
    comands = db.relationship('Comands', backref=db.backref('results', lazy=True))
    subject_id = db.Column(db.Integer, db.ForeignKey('subject.id'), nullable=False)
    subject = db.relationship('Subject', backref=db.backref('results', lazy=True))
    
class Comands(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)

class Scores(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    result_id = db.Column(db.Integer, db.ForeignKey('results.id'), nullable=False)
    result = db.relationship('Results', backref=db.backref('scores', lazy=True))
    
    visiting = db.Column(db.Float, default=0.0)
    scores_before = db.Column(db.Float, default=0.0)
    scores_test = db.Column(db.Float, default=0.0)
    test_number = db.Column(db.Integer, nullable=False)

In [11]:
import tkinter.filedialog as tkFileDialog
def open_file():
    return tkFileDialog.askopenfilename( title='Выберите файл:')
import pandas as pd

data = pd.read_csv(open_file(), delimiter=";", encoding='cp1251', decimal=",") #delimiter=";", 
data = data.dropna()
data

,Команда,Баллы,Сдал(-а),Посещение до Контрольная работа 1,Контрольная работа 1,Посещение до Контрольная работа 2,Контрольная работа 2,Посещение до Контрольная работа 3,Контрольная работа 3
0,02.03.03 Математическое обеспечение и админист...,22.6,0,0.36,1.0,0.2,0.0,0.25,0.0
1,40.03.01 Юриспруденция,76.14,1,0.64,9.1,0.8,13.04,0.75,11.0
2,10.05.01 Компьютерная безопасность,3.0,0,0.91,3.0,0.8,0.0,0.12,0.0
3,10.03.01 Информационная безопасность,22.8,0,0.82,1.9,1.0,1.7,1.0,5.7
4,10.05.01 Компьютерная безопасность,5.8,0,0.45,3.8,0.2,0.0,0.12,0.0
...,...,...,...,...,...,...,...,...,...
1043,09.03.02 Информационные системы и технологии,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1044,09.03.02 Информационные системы и технологии,76.14,1,0.91,9.81,1.0,9.33,0.88,15.0
1045,06.05.01 Биоинженерия и биоинформатика,76.62,1,0.64,7.96,0.4,11.99,0.88,11.67
1046,09.03.02 Информационные системы и технологии,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
with app.app_context() as ctx:
    ctx.push()
    db.create_all()

In [12]:
subject_temp = Subject(name="Программирование и основы алгоритмизации")
with app.app_context():
    db.session.add_all([subject_temp])
    db.session.commit()

In [13]:
exams = [name  for name in data.columns.to_list() if name.startswith("Контрольная")]
with app.app_context():
    for index, d in data.iterrows():
        comand = db.session.query(Comands).filter_by(name=d["Команда"] ).first()
        if not bool(comand):
            comand = Comands(name=d["Команда"])
            db.session.add_all([comand])
            db.session.commit()

        result = Results(succes_done= d["Сдал(-а)"]==1.0, score=d[f"Баллы"], comands=comand,subject=subject_temp, )
        db.session.add_all([result])
        db.session.commit()
        for ex in exams:
            score = Scores(result=result, visiting=d[f"Посещение до {ex}"], scores_test = d[ex], test_number=exams.index(ex)+1)
            if (f"Баллы до {ex}" in data.columns):
                score.scores_before = d[f"Баллы до {ex}"]
            db.session.add_all([score])
            db.session.commit()